In [370]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import root_mean_squared_error, accuracy_score
import xgboost as xgb
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Cleaning

In [356]:
filename = "./data/raw/properties.csv"
df = pd.read_csv(filename)
df.head()

,id,price,property_type,subproperty_type,region,province,locality,zip_code,latitude,longitude,...,fl_garden,garden_sqm,fl_swimming_pool,fl_floodzone,state_building,primary_energy_consumption_sqm,epc,heating_type,fl_double_glazing,cadastral_income
0,34221000,225000.0,APARTMENT,APARTMENT,Flanders,Antwerp,Antwerp,2050,51.217172,4.379982,...,0,0.0,0,0,MISSING,231.0,C,GAS,1,922.0
1,2104000,449000.0,HOUSE,HOUSE,Flanders,East Flanders,Gent,9185,51.174944,3.845248,...,0,0.0,0,0,MISSING,221.0,C,MISSING,1,406.0
2,34036000,335000.0,APARTMENT,APARTMENT,Brussels-Capital,Brussels,Brussels,1070,50.842043,4.334543,...,0,0.0,0,1,AS_NEW,NaN,MISSING,GAS,0,NaN
3,58496000,501000.0,HOUSE,HOUSE,Flanders,Antwerp,Turnhout,2275,51.238312,4.817192,...,0,0.0,0,1,MISSING,99.0,A,MISSING,0,NaN
4,48727000,982700.0,APARTMENT,DUPLEX,Wallonia,Walloon Brabant,Nivelles,1410,NaN,NaN,...,1,142.0,0,0,AS_NEW,19.0,A+,GAS,0,NaN


In [357]:
df.dtypes

id                                  int64
price                             float64
property_type                      object
subproperty_type                   object
region                             object
province                           object
locality                           object
zip_code                            int64
latitude                          float64
longitude                         float64
construction_year                 float64
total_area_sqm                    float64
surface_land_sqm                  float64
nbr_frontages                     float64
nbr_bedrooms                      float64
equipped_kitchen                   object
fl_furnished                        int64
fl_open_fire                        int64
fl_terrace                          int64
terrace_sqm                       float64
fl_garden                           int64
garden_sqm                        float64
fl_swimming_pool                    int64
fl_floodzone                      

In [358]:
df["fl_furnished"] = df["fl_furnished"].astype("bool")
df["fl_open_fire"] = df["fl_open_fire"].astype("bool")
df["fl_terrace"] = df["fl_terrace"].astype("bool")
df["fl_garden"] = df["fl_garden"].astype("bool")
df["fl_swimming_pool"] = df["fl_swimming_pool"].astype("bool")
df["fl_floodzone"] = df["fl_floodzone"].astype("bool")
df["fl_double_glazing"] = df["fl_double_glazing"].astype("bool")

In [359]:
df.equipped_kitchen.value_counts()

equipped_kitchen
MISSING               33022
INSTALLED             19373
HYPER_EQUIPPED         9058
SEMI_EQUIPPED          5119
USA_HYPER_EQUIPPED     4092
NOT_INSTALLED          2676
USA_INSTALLED          1662
USA_SEMI_EQUIPPED       417
USA_UNINSTALLED          92
Name: count, dtype: int64

In [360]:
kitchen_mapping = {
    'USA_INSTALLED': 'EQUIPPED',
    'INSTALLED': 'EQUIPPED',
    'USA_HYPER_EQUIPPED': 'HYPER_EQUIPPED',
    'USA_SEMI_EQUIPPED': 'SEMI_EQUIPPED',
    'USA_UNINSTALLED': 'NOT_INSTALLED'
}

df['equipped_kitchen'] = df['equipped_kitchen'].replace(kitchen_mapping)

In [361]:
df.state_building.value_counts()

state_building
MISSING           26643
GOOD              21737
AS_NEW            14736
TO_RENOVATE        5283
TO_BE_DONE_UP      4293
JUST_RENOVATED     2483
TO_RESTORE          336
Name: count, dtype: int64

In [362]:
state_building_mapping = {
    'GOOD': 'GOOD_AS_NEW',
    'AS_NEW': 'GOOD_AS_NEW',
    'JUST_RENOVATED': 'GOOD_AS_NEW',
    'TO_RENOVATE': 'REQUIRES_RENOVATION',
    'TO_BE_DONE_UP': 'REQUIRES_RENOVATION',
    'TO_RESTORE': 'REQUIRES_RENOVATION'
}

df['state_building'] = df['state_building'].replace(state_building_mapping)

In [363]:
df.epc.value_counts()

epc
MISSING    24003
B          10483
C           8953
D           7531
A           7502
F           6607
E           5452
G           3537
A+           818
A++          625
Name: count, dtype: int64

In [364]:
df.heating_type.value_counts()

heating_type
GAS         31875
MISSING     31444
FUELOIL      7711
ELECTRIC     3621
PELLET        499
WOOD          147
SOLAR         134
CARBON         80
Name: count, dtype: int64

In [365]:
print(df.isnull().sum())

id                                    0
price                                 0
property_type                         0
subproperty_type                      0
region                                0
province                              0
locality                              0
zip_code                              0
latitude                          14098
longitude                         14098
construction_year                 33391
total_area_sqm                     7615
surface_land_sqm                  36256
nbr_frontages                     26346
nbr_bedrooms                          0
equipped_kitchen                      0
fl_furnished                          0
fl_open_fire                          0
fl_terrace                            0
terrace_sqm                       13140
fl_garden                             0
garden_sqm                         2939
fl_swimming_pool                      0
fl_floodzone                          0
state_building                        0


In [366]:
df.duplicated().any()

np.False_

In [367]:
with open("data/cleaned/properties.csv", "wb") as csv_file:
    pd.DataFrame.to_csv(df, csv_file, index=False)

# Preprocessing

In [374]:
filename = "./data/cleaned/properties.csv"
df = pd.read_csv(filename)
df.columns

Index(['id', 'price', 'property_type', 'subproperty_type', 'region',
       'province', 'locality', 'zip_code', 'latitude', 'longitude',
       'construction_year', 'total_area_sqm', 'surface_land_sqm',
       'nbr_frontages', 'nbr_bedrooms', 'equipped_kitchen', 'fl_furnished',
       'fl_open_fire', 'fl_terrace', 'terrace_sqm', 'fl_garden', 'garden_sqm',
       'fl_swimming_pool', 'fl_floodzone', 'state_building',
       'primary_energy_consumption_sqm', 'epc', 'heating_type',
       'fl_double_glazing', 'cadastral_income'],
      dtype='object')

In [373]:
kitchen_order = ["NOT_INSTALLED", "SEMI_EQUIPPED", "EQUIPPED", "HYPER_EQUIPPED"]
ordinal_encoder = OrdinalEncoder(categories=[kitchen_order])

df["equipped_kitchen"] = ordinal_encoder.fit_transform(df[["equipped_kitchen"]])

ValueError: Found unknown categories ['MISSING'] in column 0 during fit

In [214]:
label_encoder = LabelEncoder()

df['property_type'] = label_encoder.fit_transform(df['property_type'])
df['zip_code'] = label_encoder.fit_transform(df['zip_code'])

In [215]:
ohe_encoder = OneHotEncoder(drop='first', sparse_output=False)

column_transformer = ColumnTransformer(
    transformers=[
        ('ohe', ohe_encoder, ["state_building", "equipped_kitchen"])
    ],
    remainder='passthrough'
).set_output(transform="pandas")

df = column_transformer.fit_transform(df)

In [349]:
df.dtypes

id                                  int64
price                             float64
property_type                      object
subproperty_type                   object
region                             object
province                           object
locality                           object
zip_code                            int64
latitude                          float64
longitude                         float64
construction_year                 float64
total_area_sqm                    float64
surface_land_sqm                  float64
nbr_frontages                     float64
nbr_bedrooms                      float64
equipped_kitchen                   object
fl_furnished                         bool
fl_open_fire                         bool
fl_terrace                           bool
terrace_sqm                       float64
fl_garden                            bool
garden_sqm                        float64
fl_swimming_pool                     bool
fl_floodzone                      

In [355]:
X = df.copy()
X.pop("price")
y = df[["price"]]

In [218]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=41, test_size=0.2)
print(X_train.shape)

(6940, 20)


In [219]:
scaler = MinMaxScaler()

X_train[["remainder__construction_year", "remainder__total_area_sqm",
        "remainder__fl_furnished", "remainder__fl_open_fire", "remainder__terrace_sqm",
        "remainder__garden_sqm", "remainder__fl_swimming_pool", "remainder__fl_floodzone", 
        "remainder__primary_energy_consumption_sqm", "remainder__fl_double_glazing"]] = scaler.fit_transform(X_train[["remainder__construction_year", "remainder__total_area_sqm",
        "remainder__fl_furnished", "remainder__fl_open_fire", "remainder__terrace_sqm",
        "remainder__garden_sqm", "remainder__fl_swimming_pool", "remainder__fl_floodzone", 
        "remainder__primary_energy_consumption_sqm", "remainder__fl_double_glazing"]])
y_train[["remainder__price"]] = scaler.fit_transform(y_train[["remainder__price"]])

# Models

## Linear Regression

In [225]:
regressor = LinearRegression()

In [226]:
regressor.fit(X_train, y_train)

LinearRegression()

In [227]:
regressor.score(X_train, y_train)

0.3756079015239221

In [228]:
regressor.predict(X_test)

array([[ 88.39166717],
       [246.10736505],
       [115.31791307],
       ...,
       [ 98.2752732 ],
       [297.64718945],
       [113.75957498]])

In [229]:
regressor.score(X_test, y_test)

-1.4425564687914112

## XGBoost

In [220]:
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)

In [221]:
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [222]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

In [223]:
train_rmse = root_mean_squared_error(y_train, y_pred_train)
test_rmse = root_mean_squared_error(y_test, y_pred_test)

print(f"Train RMSE: {train_rmse}")
print(f"Test RMSE: {test_rmse}")

Train RMSE: 0.009172488993302148
Test RMSE: 530144.4742530942


## Decision Tree

In [248]:
dtree = DecisionTreeRegressor(random_state=32)

In [249]:
dtree.fit(X_train, y_train)

DecisionTreeRegressor(random_state=32)

In [256]:
y_pred = dtree.predict(X_test)

In [257]:
rmse = root_mean_squared_error(y_test, y_pred)
print(f"Root Mean Squared Error: {rmse:.2f}")

Root Mean Squared Error: 530144.40


## Random Forest

In [254]:
forest = RandomForestRegressor(random_state=324)

In [255]:
forest.fit(X_train, y_train)

c:\Users\zelim\Desktop\becode_projects\immo-eliza-ml\.venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(random_state=324)

In [258]:
y_pred = forest.predict(X_test)

In [259]:
rmse = root_mean_squared_error(y_test, y_pred)
print(f"Root Mean Squared Error: {rmse:.2f}")

Root Mean Squared Error: 530144.55
